In [82]:
import os, sys, requests, json, pathlib, datetime, shapely
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import r5py
import geocube
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
os.chdir('C:/Users/tgwon/wri/cif/cities-cif')
sys.path.append('C:/Users/tgwon/wri/cif/cities-cif')
os.environ['GOOGLE_APPLICATION_USER'] = 'script-service-account@wri-gee.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tgwon/.google/credkey.json'
sys.path = ['C:\\Users\\tgwon\\wri\\indicators', 'C:\\Users\\tgwon\\wri\\cif\\cities-cif', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\python310.zip', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\DLLs', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\win32', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\win32\\lib', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\Pythonwin',]

from city_metrix.layers import WorldPop, UrbanExtents
from city_metrix.layers.layer_geometry import GeoExtent

In [2]:
points = gpd.GeoDataFrame.from_file('C:/Users/tgwon/wri/indicators/amenitypoints/points__public_open_space__ARG-Buenos_Aires__urbextbound.geojson')

In [3]:
CITYDATA_URL = 'https://cities-data-api.wri.org/cities'
citydata = requests.get(CITYDATA_URL).json()
focal_cities = [c for c in citydata['cities'] if c['name']=='Buenos Aires']
city = focal_cities[0]

In [5]:
url = city['layers_url']['geojson']
gpd.GeoDataFrame.from_file(url)
city_admin = gpd.GeoDataFrame.from_file(url)

Skipping field bbox: unsupported OGR type: 3
Skipping field bbox: unsupported OGR type: 3


In [6]:
the_city_urbext = UrbanExtents().get_data(GeoExtent(city_admin.total_bounds)).to_crs('EPSG:4326').dissolve()

In [7]:
bbox = GeoExtent(the_city_urbext.total_bounds)
utm_crs = bbox.as_utm_bbox().crs
worldpop_data = WorldPop(agesex_classes=[]).get_data(bbox)
wp_df = worldpop_data.drop_vars(['time']).to_dataframe().reset_index()
pop_points = gpd.GeoDataFrame(wp_df.population, geometry=gpd.points_from_xy(wp_df.x,wp_df.y))
pop_points_geogr = pop_points.set_crs(utm_crs).to_crs('EPSG:4326')
pop_points_clipped = pop_points_geogr.loc[pop_points_geogr.intersects(the_city_urbext.dissolve().geometry[0])]

Extracting layer world pop from Google Earth Engine for bbox :
[########################################] | 100% Completed | 3.46 sms


In [8]:
pop_points_clipped

,population,geometry
15878,3.872722,POINT (-58.09357 -35.04939)
15879,4.176632,POINT (-58.09247 -35.0494)
15880,4.160919,POINT (-58.09137 -35.04941)
15881,3.464365,POINT (-58.09028 -35.04941)
15882,3.208935,POINT (-58.08918 -35.04942)
...,...,...
908804,6.233791,POINT (-58.78115 -34.29107)
908805,5.842262,POINT (-58.78006 -34.29109)
909872,6.098891,POINT (-58.78765 -34.29008)
909873,6.265943,POINT (-58.78656 -34.29009)


In [9]:
b = r5py.TransportNetwork(pathlib.Path('C:/Users/tgwon/wri/indicators/pbf/ARG-Buenos_Aires__urbextbound.osm.pbf'))

In [10]:
points

,index,id,amenityclass_road,geometry
0,0,0,True,POINT (-58.38529 -34.60049)
1,1,1,True,POINT (-58.38488 -34.60157)
2,2,2,True,POINT (-58.38455 -34.60046)
3,3,3,True,POINT (-58.38451 -34.60263)
4,4,4,True,POINT (-58.38433 -34.60054)
...,...,...,...,...
23589,31699,31699,True,POINT (-58.4904 -34.57111)
23590,31700,31700,True,POINT (-58.4898 -34.57166)
23591,31704,31704,True,POINT (-57.94129 -34.90402)
23592,31705,31705,True,POINT (-57.94122 -34.90409)


In [ ]:
%%time
res = pd.DataFrame({'to_id': pop_points_clipped.index, 'numpixels': [0] * len(pop_points_clipped)})
for i in range(len(points)):
    if i % 1 == 0:
        print(i, end=' ', flush=True)
    d = r5py.TravelTimeMatrix(transport_network=b, origins=gpd.GeoDataFrame({'id': [i], 'geometry': points.iloc[[i]].centroid}), destinations=gpd.GeoDataFrame({'id': range(len(pop_points_clipped)), 'geometry': pop_points_clipped.geometry}), transport_modes=[r5py.TransportMode.WALK], max_time=datetime.timedelta(minutes=15), max_time_walking=datetime.timedelta(minutes=15))
    res.numpixels = res.numpixels + ((d.travel_time / d.travel_time).fillna(0))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 

In [144]:
%%time
pop_points['accessible_points'] = 0
pop_points.loc[pop_points_clipped.index, 'accessible_points'] = res['numpixels']
pop_points.accessible_points = pop_points.accessible_points.fillna(0).astype(int)

CPU times: total: 46.9 ms
Wall time: 43.1 ms


In [165]:
geo_grid = make_geocube(
    vector_data=pop_points.set_crs(worldpop_data.crs),
    measurements=['accessible_points'],
    like = worldpop_data,
    rasterize_function=rasterize_points_griddata,
)
geo_grid.rio.to_raster('C:/Users/tgwon/wri/indicators/ba_test.tif', engine='GeoTIFF')